In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import glob
import gc
import pandas as pd
import numpy as np
import lightgbm as lgb
import datetime
from functools import partial
from itertools import product
from multiprocessing import Pool, cpu_count
from contextlib import closing

%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, precision_recall_curve, roc_curve, average_precision_score
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, merge, Reshape, Dropout, Input, Flatten, Concatenate
from keras.layers.embeddings import Embedding
from keras.callbacks import EarlyStopping
from sklearn import metrics, ensemble, neighbors, linear_model, tree, model_selection

In [ ]:
trades_df = pd.read_csv('../input/Trade.csv')
submission_df = pd.read_csv('../input/Challenge_20180423.csv')
isin_df = pd.read_csv('../input/Isin.csv')
customer_df = pd.read_csv('../input/Customer.csv')
market_df = pd.read_csv('../input/Market.csv')

trades_df = trades_df.rename({'TradeDateKey': 'DateKey'}, axis=1)
trades_df.DateKey = pd.to_datetime(trades_df.DateKey, format='%Y%m%d')
trades_df = trades_df.sort_values(['DateKey'])

trades_df['WeekDateKey'] = trades_df.DateKey - pd.to_timedelta(trades_df.DateKey.dt.dayofweek, 'd')
trades_df = trades_df.drop_duplicates(['CustomerIdx', 'IsinIdx', 'BuySell', 'WeekDateKey', 'CustomerInterest'])

trades_df1 = trades_df[trades_df.CustomerInterest == 1]

#trades_df.shape

ngs = []
for i in [1, 4, 12, 16, 24]:
    tmp = trades_df1[['CustomerIdx', 'IsinIdx', 'BuySell', 'WeekDateKey']].copy()
    tmp.WeekDateKey += pd.to_timedelta(7 * i, 'd')
    ngs.append(tmp)

negative_sampling = pd.concat(ngs).reset_index(drop=True)
negative_sampling = negative_sampling[negative_sampling.WeekDateKey < '2018-04-23']
negative_sampling = negative_sampling.drop_duplicates(['CustomerIdx', 'IsinIdx', 'BuySell', 'WeekDateKey'])
#negative_sampling.shape

negative_sampling = pd.concat([
    negative_sampling,
    trades_df1[['CustomerIdx', 'IsinIdx', 'BuySell', 'WeekDateKey', 'CustomerInterest']]
])

reord1 = negative_sampling[negative_sampling.duplicated(['CustomerIdx', 'IsinIdx', 'BuySell', 'WeekDateKey'], keep=False)
    &
    (negative_sampling.CustomerInterest == 1)
]
reord1['CustomerInterest'] = 1

reord0 = negative_sampling[~negative_sampling.duplicated(['CustomerIdx', 'IsinIdx', 'BuySell', 'WeekDateKey'], keep=False)
    &
    (negative_sampling.CustomerInterest == 1)
]
reord0['CustomerInterest'] = 0

reordered = pd.concat([reord0, reord1])

trades_df = reordered
del reordered
#gc.collect()

trades_df.WeekDateKey = pd.to_datetime(trades_df.WeekDateKey)

submission_df = submission_df.rename({'DateKey': 'WeekDateKey'}, axis=1)
submission_df.WeekDateKey = pd.to_datetime(submission_df.WeekDateKey, format='%Y%m%d')

market_df = market_df.rename({'DateKey': 'WeekDateKey'}, axis=1)
market_df.WeekDateKey = pd.to_datetime(market_df.WeekDateKey, format='%Y%m%d')
market_df.WeekDateKey = market_df.WeekDateKey + pd.to_timedelta(7 - market_df.WeekDateKey.dt.dayofweek, 'd')
market_df.Price = np.log(market_df.Price, dtype=np.float32)
market_df = market_df.groupby(['WeekDateKey', 'IsinIdx']).mean().reset_index()

isin_df = pd.read_csv("../input/Isin.csv")
isin_df.fillna("missing", inplace=True)
isin_df = isin_df.rename({'Region': 'BondRegion'}, axis=1)
for col in ['ActualMaturityDateKey', 'IssueDateKey']:
    isin_df[col] = pd.to_datetime(isin_df[col], format='%Y%m%d')
    isin_df[col] = isin_df[col].apply(lambda x: x.toordinal())

categorical_isin = list(isin_df.dtypes[isin_df.dtypes == 'object'].index) + ['TickerIdx']
# for col in categorical_isin:
#     isin_df[col] =isin_df[col].astype('category')

customer_df = pd.read_csv("../input/Customer.csv")
customer_df.fillna("missing", inplace=True)
customer_df = customer_df.rename({'Region': 'CustomerRegion'}, axis=1)
categorical_customer = list(customer_df.dtypes[customer_df.dtypes == 'object'].index)
# for col in categorical_customer:
#     customer_df[col] = customer_df[col].astype('category')

categorical_features = categorical_customer + categorical_isin

trades_df = trades_df.sort_values('WeekDateKey')

alldata_df = pd.concat([trades_df, submission_df], sort=False).reset_index(drop=True)

del trades_df
gc.collect()

market_df = pd.merge(market_df, isin_df, on='IsinIdx', how='left')
market_df['Order_dom'] = market_df['WeekDateKey'].apply(lambda d: (d.day-1) // 7 + 1)
market_df['Order_month'] = market_df['WeekDateKey'].dt.month

def rolling_handler(x, feature, rolling=-1):
    rolling = len(x) if rolling == -1 else rolling
    return getattr(x.rolling(rolling, 1), feature)()

features = ['sum', 'min', 'max', 'mean']



In [ ]:
for groupby in categorical_isin + ['Order_dom', 'Order_month']:
    groupby = ['IsinIdx'] + [groupby]
    suffix = '_&_'.join(groupby)
    market_df.sort_values(groupby + ['WeekDateKey'], inplace=True)
    start_time = datetime.datetime.now()
    
    def worker_func(grouped, feature):
        return grouped.apply(lambda x: rolling_handler(x, feature))
    
    grouped = market_df[groupby + ['Price']].groupby(groupby, sort=False).Price
    partial_worker_func = partial(worker_func, grouped)
    
    with closing(Pool(24)) as p:
        ret_list = p.map(partial_worker_func, features)
        p.terminate()
    
    for ret, feature in zip(ret_list, features):
        name = '{}_{}_by_{}'.format('Price', feature, suffix)
        market_df[name] = ret
    
    del ret_list, grouped
    gc.collect()
    print(suffix, (datetime.datetime.now() - start_time).total_seconds() / 60)

In [ ]:
trades_df = pd.concat([reord0, reord1])
alldata_df = pd.concat([trades_df, submission_df], sort=False).reset_index(drop=True)

In [ ]:
market_df = market_df.dropna()

alldata_df = pd.merge(alldata_df, customer_df, on='CustomerIdx', how='left')
alldata_df = pd.merge(alldata_df, market_df, on=['WeekDateKey', 'IsinIdx'], how='left')

# del isin_df, customer_df, market_df
# gc.collect()

alldata_df.loc[alldata_df.BuySell == 'Sell', 'Price'] *= -1

alldata_df['BuySell'] = alldata_df['BuySell'].map({'Buy': 0, 'Sell': 1})
categorical_features.extend(['BuySell'])

base_features = alldata_df.columns.difference(
    categorical_features + ['CustomerInterest', 'IsinIdx', 'CustomerIdx', 'WeekDateKey', 'PredictionIdx']
).tolist()
#base_features

alldata_df.drop(['TickerIdx'], axis=1, inplace=True)

for c in base_features:
    alldata_df[c].fillna(alldata_df[c].mean(), inplace=True)

train_df = alldata_df[(alldata_df.WeekDateKey >= '2016-01-01') & (alldata_df.WeekDateKey < '2018-04-09')]
valid_df = alldata_df[(alldata_df.WeekDateKey >= '2018-04-09') & (alldata_df.WeekDateKey < '2018-04-23')]
test_df = alldata_df[alldata_df.WeekDateKey >= '2018-04-23']

#train_df['Price'].isnull().value_counts()

#categorical_features.append("BuySell")
#categorical_features


In [ ]:
if "TickerIdx" in categorical_features:
    categorical_features.remove("TickerIdx")
for c in categorical_features:
    print(c)
    lbl = LabelEncoder()
    lbl.fit(list(train_df[c].values) + list(valid_df[c].values) + list(test_df[c].values))
    train_df[c] = lbl.transform(list(train_df[c].values))
    valid_df[c] = lbl.transform(list(valid_df[c].values))
    test_df[c] = lbl.transform(list(test_df[c].values))

In [ ]:
#categorical_features.remove("TickerIdx")
X_train = train_df[base_features + categorical_features]
X_valid = valid_df[base_features + categorical_features]
X_test = test_df[base_features + categorical_features]
y_train = train_df['CustomerInterest'].values
y_valid = valid_df['CustomerInterest'].values
y_test = test_df['CustomerInterest'].fillna(0).values
PredictionIdx = test_df['PredictionIdx']

In [ ]:
y_train

In [ ]:
embed_cols = categorical_features
len_embed_cols = []
for col in embed_cols:
    len_embed_cols.append(X_train[col].nunique())

In [ ]:
def build_embedding_network(len_embed_cols):
    
    model_out = []
    model_in  = []
    
    for dim in len_embed_cols:
        input_dim = Input(shape=(1,))
        embed_dim = Embedding(dim, dim//2, input_length=1)(input_dim)
        embed_dim = Dropout(0.25)(embed_dim)
        embed_dim = Reshape((dim//2,))(embed_dim)
        model_out.append(embed_dim)
        model_in.append(input_dim)
    
    input_num = Input(shape=(68,), dtype='float32')
    outputs = Concatenate(axis=1)([*model_out, input_num])
    
#     outputs = (Dense(128))(outputs) 
#     outputs = (Activation('relu'))(outputs)
#     outputs = (Dropout(.35))(outputs)
    outputs = (Dense(64))(outputs)
    outputs = (Activation('relu'))(outputs)
    outputs = (Dropout(.15))(outputs)
    outputs = (Dense(32))(outputs) 
    outputs = (Activation('relu'))(outputs)
    outputs = (Dropout(.15))(outputs)
    outputs = (Dense(1))(outputs)
    outputs = (Activation('sigmoid'))(outputs)
    
    model = Model([*model_in, input_num], outputs)

    model.compile(loss='binary_crossentropy', optimizer='adam')
    model.summary()
    
    return model

In [ ]:
def preproc(X_train, X_val, X_test):

    input_list_train = []
    input_list_val = []
    input_list_test = []
    
    #the cols to be embedded: rescaling to range [0, # values)
    for c in embed_cols:
        raw_vals = np.unique(X_train[c])
        val_map = {}
        for i in range(len(raw_vals)):
            val_map[raw_vals[i]] = i       
        input_list_train.append(X_train[c].map(val_map).values)
        input_list_val.append(X_val[c].map(val_map).fillna(0).values)
        input_list_test.append(X_test[c].map(val_map).fillna(0).values)
        
    #the rest of the columns
    other_cols = [c for c in X_train.columns if (not c in embed_cols)]
    input_list_train.append(X_train[other_cols].values)
    input_list_val.append(X_val[other_cols].values)
    input_list_test.append(X_test[other_cols].values)
    
    return input_list_train, input_list_val, input_list_test

In [ ]:
num_cols = base_features

# Fit the scaler only on train data

scaler = MinMaxScaler().fit(alldata_df[num_cols])
X_train.loc[:,num_cols] = scaler.transform(X_train[num_cols])
X_valid.loc[:,num_cols] = scaler.transform(X_valid[num_cols])
X_test.loc[:,num_cols] = scaler.transform(X_test[num_cols])

In [ ]:
for c in categorical_features:
    print(c)
    lbl = LabelEncoder()
    lbl.fit(list(train_df[c].values) + list(valid_df[c].values) + list(test_df[c].values))
    train_df[c] = lbl.transform(list(train_df[c].values))
    valid_df[c] = lbl.transform(list(valid_df[c].values))
    test_df[c] = lbl.transform(list(test_df[c].values))
    train_df[c] = train_df[c].apply(pd.to_numeric)
    valid_df[c] = valid_df[c].apply(pd.to_numeric)
    test_df[c] = test_df[c].apply(pd.to_numeric)

In [ ]:
proc_X_train_f, proc_X_val_f, proc_X_test_f = preproc(X_train, X_valid, X_test)
NN = build_embedding_network(len_embed_cols)
callbacks = [EarlyStopping(monitor='val_loss', patience=5)]
NN.fit(proc_X_train_f, y_train, epochs=50, batch_size=1024, verbose=1,callbacks=callbacks,validation_data=(proc_X_val_f, y_valid))
# val_preds = NN.predict(proc_X_val_f)
# v_auc  = roc_auc_score(val_y.values, val_preds)

In [ ]:
val_preds = NN.predict(proc_X_test_f)

In [ ]:
pd.DataFrame({'PredictionIdx': PredictionIdx, 'CustomerInterest': val_preds[:,0]}).to_csv('nn_ma_2.csv', index=None)